In [1]:
import cv2
from sklearn.cluster import KMeans
import numpy as np
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import xml.etree.ElementTree as ET

In [3]:
#Getting the dominant colours from an image.
def get_colors(image, number_of_colors):
    
    # Resizing the image
    modified_image = cv2.resize(image, (600, 400), interpolation = cv2.INTER_AREA)
    modified_image = modified_image.reshape(modified_image.shape[0]*modified_image.shape[1], 3)
    
    #Making clusters of image colours
    clf = KMeans(n_clusters = number_of_colors)
    labels = clf.fit_predict(modified_image)
    
    counts = Counter(labels)
    
    center_colors = clf.cluster_centers_
    ordered_colors = [center_colors[i]/255 for i in counts.keys()]
    rgb_colors = [ordered_colors[i]*255 for i in counts.keys()]
    
    # returning the rgb colours
    return rgb_colors

In [4]:

# This function will input an image and will return the colour of helmet by comparing the image colour with Red,blue,Yello,Green

def get_helmet_colour(image,threshold = 60, number_of_colors = 4): 
    image_colors = get_colors(image, number_of_colors)
    minimum_diff = float('inf')
    helmet_color = False
    
    # iterating through all the possible colours and finding the closest colour to the image given(helmet bbox) 
    for color_name,color_value in helmet_colors.items():
        selected_color = rgb2lab(np.uint8(np.asarray([[color_value]])))
        for i in range(number_of_colors):
            curr_color = rgb2lab(np.uint8(np.asarray([[image_colors[i]]])))
            diff = deltaE_cie76(selected_color, curr_color)

            if (diff < threshold):
                if diff < minimum_diff:
                    minimum_diff = diff
                    helmet_color = color_name
                   
    return helmet_color

In [5]:
# this function is substitute to the trained model... it will read the xml file and will return the bboxes.
def get_detections(path):
    tree = ET.parse(path)
    root = tree.getroot()

    sample_annotations = []
    
    for neighbor in root.iter('object'):
        name = neighbor.find('name').text
        xmin = int(neighbor.find('bndbox').find('xmin').text)
        ymin = int(neighbor.find('bndbox').find('ymin').text)
        xmax = int(neighbor.find('bndbox').find('xmax').text)
        ymax = int(neighbor.find('bndbox').find('ymax').text)
        
        sample_annotations.append([name,[xmin, ymin, xmax, ymax]])
    return sample_annotations

In [9]:
image_path = 'sample_dataset/images/hard_hat_workers4.png'
# We are using the xml file to extract the bbox. In place of trained model.
annotation_path = 'sample_dataset/annotations/hard_hat_workers4.xml'
output_file_name = 'sample_dataset/output/output_image4.jpg'


helmet_colors = {
    'GREEN': [0, 128, 0],
    'BLUE': [0, 0, 128],
    'YELLOW': [255, 255, 0],
    'RED':[128,0,0]
}
helmet_type = {
    'GREEN' : 'Safety Supervisor',
    'RED' : 'Contractor',
    'BLUE' : 'Supervisor',
    'YELLOW' : 'Labour'
}


detections = get_detections(annotation_path)

img = cv2.imread(image_path)
for detection in detections :
    if detection[0] == 'helmet' :
        xmin,ymin,xmax,ymax = detection[1]
        #extracting only the bbox from original image then taking only the upper half of bbox since that is where helmet will be.
        img_helmet = img[ymin:((ymin+ymax)//2),xmin:xmax]
        img_helmet = cv2.cvtColor(img_helmet, cv2.COLOR_BGR2RGB)
        # Recognising the helmet colour
        helmet_colour = get_helmet_colour(img_helmet, 60, 4)
        if not helmet_colour :
            print('Some different colour')
            helmet_colour = 'Diff'
        else :
            print('Helmet colour:',helmet_colour,'     Type:',helmet_type[helmet_colour])  
        img = cv2.putText(img,helmet_colour,(xmin,ymin), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,0), 1, cv2.LINE_AA)

cv2.imwrite(output_file_name,img)

Helmet colour: YELLOW      Type: Labour


True